# Scenario 3: Multiple DS working on multiple ML models

MLflow setup:
- Tracking server: yes, remote server (EC2)
- Backend store: postgresql database
- Artifacts store: s3 bucket

The experiments can be explored by accessing the remote server. 

The example uses AWS to host a remote server. In order to run the example you will need an AWS account. Follow the steps described in the file mlflow_on_aws.md to create an AWS account and launch the tracking server.

In [5]:
import mlflow
import os
import boto3

os.environ["AWS_PROFILE"] = "default"

TRACKING_SERVER_HOST = "ec2-3-17-153-227.us-east-2.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")


In [6]:
print(f"tracking URI: {mlflow.get_tracking_uri()}")

tracking URI: http://ec2-3-17-153-227.us-east-2.compute.amazonaws.com:5000


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-2/2', creation_time=1740872841368, experiment_id='2', last_update_time=1740872841368, lifecycle_stage='active', name='my-experiment-2', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: {mlflow.get_artifact_uri()}")


2025/03/02 00:01:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: s3://mlflow-artifacts-remote-2/2/4dfda94370024e079dbb769014546ae5/artifacts
🏃 View run handsome-yak-701 at: http://ec2-3-17-153-227.us-east-2.compute.amazonaws.com:5000/#/experiments/2/runs/4dfda94370024e079dbb769014546ae5
🧪 View experiment at: http://ec2-3-17-153-227.us-east-2.compute.amazonaws.com:5000/#/experiments/2


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-2/2', creation_time=1740872841368, experiment_id='2', last_update_time=1740872841368, lifecycle_stage='active', name='my-experiment-2', tags={}>]

## Interacting with Model Registry

In [13]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [14]:
client.search_registered_models()

[]

In [15]:
run_id = client.search_runs(["2"])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2025/03/02 00:06:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1740873968736, current_stage='None', description='', last_updated_timestamp=1740873968736, name='iris-classifier', run_id='4dfda94370024e079dbb769014546ae5', run_link='', source='s3://mlflow-artifacts-remote-2/2/4dfda94370024e079dbb769014546ae5/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>